## Моделирование стоимости бриллиантов

Цель моей работы - проанализировать базу данных, включающую в себя основные характеристики бриллиантов, с целью прогнозирования их цены с учетом различных факторов. В конечном итоге, мной будет разработана обучающая машина, способная выполнять данную функцию.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

diamonds = pd.read_csv("../input/diamonds/diamonds.csv")

## Первичное рассмотрение и проверка работы базы данных

In [ ]:
diamonds.head()

Можно попробовать вытащить еще немного информации из базы данных.

In [ ]:
diamonds.info()

Можно заметить такой показатель, как "Unnamed 0". В целом, он нам не нужен и его можно убрать.

In [ ]:
diamonds = diamonds.drop("Unnamed: 0", axis = 1)

diamonds["price"] = diamonds["price"].astype(float)

diamonds.head()

Как можно заметить, в выборка представляет из себя 53490 образцов и стоит сказать, что нет ни одной пропуска в наборе данных.

Для того, чтобы упросить работы с данными, необходимо перевести все основные категории-характеристика бриллиантов в числовые.

Сначала, в целом, надо посмотреть, какие категории присутствуют в базе данных.

In [ ]:
diamonds["cut"].value_counts()

In [ ]:
diamonds["color"].value_counts()

In [ ]:
diamonds["clarity"].value_counts()

Можно сделать вывод о том, что категорий не так много! Но это не отменяет большое количество столбцов.

Далее можно сделать основное заключение по числовым атрибутам базы данных и сделать гистограмму на их основе.


In [ ]:
diamonds.describe()

In [ ]:
diamonds.hist(bins = 50, figsize = (20, 15))
plt.show()

## Создание тест сета

Благодаря созданию тестового сета мы можем проверить производительность работы имеющейся базы данных на примере новых экземпляров. Однако перед этим необходимо провести стратификацию выборки на отдельные подгруппы, которая позволит выделить конкретные экземпляры, которые позволяю более правильно оценить набор всех данных в целом

Чтобы провести эту операцию, нам необходимо узнать атрибуты, которые будет лучше всего коррелировать с ценой бриллианта, поэтому мной будет проведено определение стандартного коэффициента корреляции по Пирсону.

In [ ]:
corr_matrix = diamonds.corr()

plt.subplots(figsize = (10, 8))
sns.heatmap(corr_matrix, annot = True)
plt.show()

Итак, какие выводы можно сделать:
- x, y, z обладают самой сильной корреляцией с price
- carat также обладает одной из самых корреляций с price (0,92)
- самая слабая корреляция наблюдается у table и depth

Теперь построим гистограмму такого атрибута как carat

In [ ]:
diamonds["carat"].hist(bins = 50)
plt.show()

Большинство алмазов примерно от 0,3 до 1,5 карат. Далее разделим их на 5 категорий, причем следующие после 5-й категории сливаются в 5-ю категорию.

In [ ]:
diamonds["carat_cat"] = np.ceil(diamonds["carat"] / 0.35)

diamonds["carat_cat"].where(diamonds["carat_cat"] < 5, 5.0, inplace = True)

Теперь можно посмотреть, сколько алмазов распределено по категориям в каратах.

In [ ]:
diamonds["carat_cat"].value_counts()

In [ ]:
diamonds["carat_cat"].hist()
plt.show()

Распределение выглядит довольно хорошо. После этого мы можем выполнить стратификационную выборку на основе категории carat при помощи функции "StratifiedShuffleSplit".

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)

for train_index, test_index in split.split(diamonds, diamonds["carat_cat"]):
    strat_train_set = diamonds.loc[train_index]
    strat_test_set = diamonds.loc[test_index]

После всех приготовлений, теперь такая переменная, как carat_cat нам уже не нужна и мы можем ее убрать

In [ ]:
for set in (strat_train_set, strat_test_set):
    set.drop(["carat_cat"], axis = 1, inplace = True)

Теперь обозначим нашу базу данных как "Stratified Train set".

In [ ]:
diamonds = strat_train_set.copy()
diamonds.head()

## Визуализация полученных данных

Теперь можно сделать некоторые визуализации и провести наблюдения из полученных рисунков.

In [ ]:
sns.pairplot(diamonds[["price", "carat", "cut"]], hue = "cut", height = 5)
plt.show()
sns.barplot(x = "carat", y = "cut", data = diamonds)
plt.show()
sns.barplot(x = "price", y = "cut", data = diamonds)
plt.show()

«Fair» огранки наиболее весомые, но они не самые дорогие алмазы. «Premium» весят меньше, чем "fair", а затем стоят дороже. «Ideal» весят намного меньше, и они наименее дороги. Таким образом, огранка относительно учитывается при определении цены алмаза.

In [ ]:
sns.pairplot(diamonds[["price", "carat", "color"]], hue = "color", height = 5)
plt.show()
sns.barplot(x = "carat", y = "color", data = diamonds)
plt.show()
sns.barplot(x = "price", y = "color", data = diamonds)
plt.show()

Здесь мы можем видеть, что цвет J, который является наиболее взвешенным, также является самым дорогим. Последние 2 сюжета очень похожи. Мы могли видеть здесь, что цвет алмаза также очень зависит от его цены.

In [ ]:
sns.pairplot(diamonds[["price", "carat", "clarity"]], hue = "clarity", height = 5)
plt.show()
sns.barplot(x = "carat", y = "clarity", data = diamonds)
plt.show()
sns.barplot(x = "price", y = "clarity", data = diamonds)
plt.show()

Здесь мы можем видеть, что I1 не обладает наивысшей четкостью, хотя он и самый дорогой. Но есть еще кое-что: кроме I1, если остальное останется, цена на бриллиант могут быть довольно относительными его ясности в некоторой степени.

## Шкалирование основных свойств

Скалирование объектов может быть выполнено двумя способами: Минимальное-максимальное скалирование и Стандартизация. Я бы предпочел использовать стандартизацию, потому что на нее гораздо меньше влияют выбросы.

In [ ]:
diamonds = strat_train_set.drop("price", axis = 1)

diamond_labels = strat_train_set["price"].copy()

diamonds_num = diamonds.drop(["cut", "color", "clarity"], axis = 1)
diamonds_num.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

num_scaler = StandardScaler()
diamonds_num_scaled = num_scaler.fit_transform(diamonds_num)

pd.DataFrame(diamonds_num_scaled).head()

Это то, как будут выглядеть наши данные в процессе обработки. Так мы сделали обучающую машину, которая позволяет прогнозировать.

## Обработка атрибутов

Теперь мы создаем один двоичный атрибут для каждой категории: один атрибут будет один, а остальные - 0. Это называется One-Hot Encoding. Scikit-Learn предоставляет кодировщик OneHotEncoder для преобразования атрибутов нашей категории в векторы One-Hot.

In [ ]:
diamonds_cat = diamonds[["cut", "color", "clarity"]]
diamonds_cat.head()


In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
diamonds_cat_encoded = cat_encoder.fit_transform(diamonds_cat)

pd.DataFrame(diamonds_cat_encoded.toarray()).head()

Как я говорил ранее, в нашей таблице, в конечном счете, будет много столбцов.

## Трансформация 

Для того, чтобы не было проблем при обработке данных, необходимоперейти к классу Scikit-Learn ColumnTransformer. Это объединение обеспечивает единый конвейер для всего набора данных.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(diamonds_num)
cat_attribs = ["cut", "color", "clarity"]

pipeline = ColumnTransformer([
    ("num", StandardScaler(), num_attribs), 
    ("cat", OneHotEncoder(), cat_attribs) 
])

In [ ]:
diamonds_ready = pipeline.fit_transform(diamonds)

pd.DataFrame(diamonds_ready).head()

Теперь у нас есть наш недавно преобразованный набор данных, который можно легко использовать в наших алгоритмах машинного обучения.

## Разработка модели и проверка ее работы

Следующим шагом я создам одну функцию, которая будет проходить через каждый алгоритм. У меня также будут переменные, которые содержат результаты алгоритмов для будущих сравнений. 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from random import randint

X_test = strat_test_set.drop("price", axis = 1)
y_test = strat_test_set["price"].copy()

models_rmse = []
cvs_rmse_mean = []
tests_rmse = []
tests_accuracy = []
models = []

def display_model_performance(model_name, model, diamonds = diamonds_ready, labels = diamond_labels,
                              models_rmse = models_rmse, cvs_rmse_mean = cvs_rmse_mean, tests_rmse = tests_rmse,
                              tests_accuracy = tests_accuracy, pipeline = pipeline, X_test = X_test,
                              y_test = y_test, cv = True):

    model.fit(diamonds, labels)
    
    predictions = model.predict(diamonds)
    
    model_mse = mean_squared_error(labels, predictions)
    model_rmse = np.sqrt(model_mse)
    
    cv_score = cross_val_score(model, diamonds, labels, scoring = "neg_mean_squared_error", cv = 10)
    cv_rmse = np.sqrt(-cv_score)
    cv_rmse_mean = cv_rmse.mean()
    
    print("RMSE: %.4f" %model_rmse)
    models_rmse.append(model_rmse)
    
    print("CV-RMSE: %.4f" %cv_rmse_mean)
    cvs_rmse_mean.append(cv_rmse_mean)
    
    print("--- Test Performance ---")
    
    X_test_prepared = pipeline.transform(X_test)
    
    model.fit(X_test_prepared, y_test)
    
    test_predictions = model.predict(X_test_prepared)
    
    test_model_mse = mean_squared_error(y_test, test_predictions)
    test_model_rmse = np.sqrt(test_model_mse)
    print("RMSE: %.4f" %test_model_rmse)
    tests_rmse.append(test_model_rmse)
    
    test_accuracy = round(model.score(X_test_prepared, y_test) * 100, 2)
    print("Accuracy:", str(test_accuracy)+"%")
    tests_accuracy.append(test_accuracy)
    
    start = randint(1, len(y_test))
    some_data = X_test.iloc[start:start + 7]
    some_labels = y_test.iloc[start:start + 7]
    some_data_prepared = pipeline.transform(some_data)
    print("Predictions:\t", model.predict(some_data_prepared))
    print("Labels:\t\t", list(some_labels))
    
    models.append(model_name)
    
    plt.scatter(diamond_labels, model.predict(diamonds_ready))
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    x_lim = plt.xlim()
    y_lim = plt.ylim()
    plt.plot(x_lim, y_lim, "k--")
    plt.show()
    
    print("------- Test -------")
    plt.scatter(y_test, model.predict(X_test_prepared))
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.plot(x_lim, y_lim, "k--")
    plt.show()

Теперь мы можем приступить к подгонке моделей и получить ошибку их производительности. Помним, что для оценки эффективности мы используем среднеквадратическую ошибку.

Начнем с самой простой модели - «Линейная регрессия»

### Линейная регрессия

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression(normalize = True)
display_model_performance("Linear Regression", lin_reg)

### Регрессия "Случайный лес"

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators = 10, random_state = 42)
display_model_performance("Random Forest Regression", forest_reg)

### Регрессия "Дерево решений"

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state = 42)
display_model_performance("Decision Tree Regression", tree_reg)

## Сравнение ранее рассмотренных регрессий 

In [ ]:
compare_models = pd.DataFrame({ "Algorithms": models, "Models RMSE": models_rmse, "CV RMSE Mean": cvs_rmse_mean,
                              "Tests RMSE": tests_rmse, "Tests Accuracy": tests_accuracy })
compare_models.sort_values(by = "Tests Accuracy", ascending = False)

Хорошо. Стоит обратить внимание, что вытекает точность 100% из модели регрессии "Дерево решений". Это слишком идеально. Стоит отметить, что некоторые наборы данных из набора тестов были выбраны и сравнены, возможно, это и правильно. Другая модель, от которой мы могли бы зависеть, - это «Случайный Лес». Это работает относительно хорошо, на мой взгляд.

In [ ]:
sns.barplot(x = "Tests Accuracy", y = "Algorithms", data = compare_models)
plt.show()

## Сохранение модели

In [ ]:
import pickle

with open('final_model.pkl', 'wb') as f:
    pickle.dump(tree_reg, f)

## Заключение

Алгоритм "Дерево решений" выигрывает здесь!